# User behavioral analysis - Swiss top job portal

FOR STREAMLIT DASHBOARD, USE PYTHON SCRIPT "User_behavioral_analysis_dashboard.py"
- 1) Overview
- 2) Exploratory Data Analysis
    - 2.1) How many clicks does a user make? In how many different regions?
    - 2.2) Which are the most clicked jobs, sectors and work regions?
    - 2.3) Are there differences between logged-in users and anonymous sessions?

## 1) Overview datasets and merge

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
# Load clicks dataset
df_c = pd.read_csv('jobs_ad_clicks.csv')

print('Number of records:', df_c.shape[0], '\n')
print('Number of unique user IDs:', df_c['userId'].nunique(), '\n')
print('Number of unique job IDs:', df_c['jobId'].nunique())
df_c.head()

Number of records: 6680772 

Number of unique user IDs: 852775 

Number of unique job IDs: 182706


,userId,jobId,dateYM,nsession,loggedIn
0,00000000-2966-46d5-ffff-ffffc41afe34,7b660fb5-569c-4437-8a46-017385741663,2021-12-06,1,False
1,00000000-2966-bb95-ffff-ffffb02a6c56,168d7a4f-97ff-40a6-9ec7-0e35b6cf2506,2021-12-08,1,False
2,00000000-2966-bb95-ffff-ffffb02a6c56,1716c667-8586-441e-99a9-16fb7b2f481d,2021-12-08,1,False
3,00000000-2966-bb95-ffff-ffffb02a6c56,2d269efd-72c0-479e-90c3-97221b744e2e,2021-12-07,1,False
4,00000000-2966-bb95-ffff-ffffb02a6c56,4516bb7f-0b34-4536-bb2b-510838de85fc,2021-12-08,1,False


In [3]:
# Load job ads details dataset
df_j = pd.read_csv('dfmeta_jobs_filtered_v2.csv', encoding= 'unicode_escape')

print('Number of job IDs:', df_j.shape[0], '\n')
df_j.head()

Number of job IDs: 67023 



,jobId,place,zip,title,sector,workregion,pub_datum
0,36e2eb59-8656-4d25-93d7-8ee388341ea4,Zug / Luzern / Muri / Rotkreuz / Zürich / Cham,NaN,"Käseexperte als Leiter Beschaffung, Mitglied d...",jobsse-21,:11:1:,2021-10-25 08:32:42
1,ca13a54a-7b11-431f-a673-8ac209f9c8f8,Buchs AG,5033,Category Manager (m/w/d),jobsse-0,:8:,2021-10-19 10:32:29
2,a8bf513f-ded4-40c9-9145-fa9de15df6fd,Zürich,8001,AssistentIn Leitung Stadtärztlicher Dienst,jobsse-16,:3:,2021-10-29 13:34:30
3,a71b29ed-c2ca-4a22-9143-5a928002587d,Gland,1196,MECANICIEN/NE VELO (F/H),jobsse-0,:19:,2021-10-12 18:32:43
4,583ff476-41c9-4229-8958-e24f1f507058,Bern,3027,SAP Integrations- und Change-Spezialistin/-Spe...,jobsse-16,:9:,2021-10-20 05:02:52


### Merge dataset on job ID column
After merging the two datasets on the job ID column, the final dataset available for the analysis is made of:
- 5'278'368 records (unique job ad clicks per user)
- 63'832 unique job ads
- 782'084 unique users
- Ads publication dates range between April 1st, 2021 and December 14, 2021 
- Ads click dates span between October 28th, 2021 and December 15, 2021

In [4]:
# Merge two datasets
df = df_c.merge(df_j, how='inner', on='jobId')

# Assign correct datatypes
df['dateYM'] = pd.to_datetime(df['dateYM'])
df['pub_datum'] = pd.to_datetime(df['pub_datum'])
df['place'] = df['place'].astype('category')
df['sector'] = df['sector'].astype('category')


# Print basic details
print('Number of records:', df.shape[0], '\n')
print('Number of unique user IDs:', df['userId'].nunique(), '\n')
print('Number of unique job IDs:', df['jobId'].nunique(), '\n')
print('Ads click dates range:', df['dateYM'].min(), '-', df['dateYM'].max(), '\n')
print('Ads publication dates range:', df['pub_datum'].min(), '-', df['pub_datum'].max(), '\n')
print('Missing data by column:', '\n', df.isnull().sum(), '\n')

df.head()

Number of records: 5278368 

Number of unique user IDs: 782084 

Number of unique job IDs: 63832 

Ads click dates range: 2021-10-28 00:00:00 - 2021-12-15 00:00:00 

Ads publication dates range: 2021-04-01 14:18:42 - 2021-12-14 00:03:55 

Missing data by column: 
 userId              0
jobId               0
dateYM              0
nsession            0
loggedIn            0
place          278566
zip           1427734
title          277882
sector         277882
workregion     277909
pub_datum      277882
dtype: int64 



,userId,jobId,dateYM,nsession,loggedIn,place,zip,title,sector,workregion,pub_datum
0,00000000-2966-46d5-ffff-ffffc41afe34,7b660fb5-569c-4437-8a46-017385741663,2021-12-06,1,False,Region Zürich,NaN,Divisional Head Application Suite (w/m) - ID 6...,jobsse-13,:5:11:,2021-12-06 09:02:20
1,00cfb878-2848-49b9-a0da-90a8c73b388b,7b660fb5-569c-4437-8a46-017385741663,2021-12-06,3,False,Region Zürich,NaN,Divisional Head Application Suite (w/m) - ID 6...,jobsse-13,:5:11:,2021-12-06 09:02:20
2,02628c72-e19b-4277-b7e5-5246bdbd0cd7,7b660fb5-569c-4437-8a46-017385741663,2021-12-06,1,False,Region Zürich,NaN,Divisional Head Application Suite (w/m) - ID 6...,jobsse-13,:5:11:,2021-12-06 09:02:20
3,09550379-cbcc-44cc-9390-56251cbd76cf,7b660fb5-569c-4437-8a46-017385741663,2021-12-06,1,False,Region Zürich,NaN,Divisional Head Application Suite (w/m) - ID 6...,jobsse-13,:5:11:,2021-12-06 09:02:20
4,0fe39d9d-1c20-443e-9512-bac0686e6563,7b660fb5-569c-4437-8a46-017385741663,2021-12-07,1,False,Region Zürich,NaN,Divisional Head Application Suite (w/m) - ID 6...,jobsse-13,:5:11:,2021-12-06 09:02:20


### Missing values
Approximately 5% (277'882/5'278'368) of records lacks info about all the job ad details and can therefore be dropped from the sample before the analysis.

In [ ]:
# Visualize patterns in missing values present in the data
plt.figure(figsize=(18,8))
sns.heatmap(df.isnull(), cbar=False)
plt.ylabel('Index number', fontweight='light', fontsize=12)
plt.xticks(rotation=45, ha="right")
plt.title('Patterns in missing values (white dashes)')
plt.show()

In [5]:
# Drop observations missing job ads details (5% of total sample)
df = df.dropna(thresh=df.shape[1]*0.66).copy()

# Print basic details
print('Number of records:', df.shape[0], '\n')
print('Number of unique user IDs:', df['userId'].nunique(), '\n')
print('Number of unique job IDs:', df['jobId'].nunique(), '\n')
print('Ads click dates range:', df['dateYM'].min(), '-', df['dateYM'].max(), '\n')
print('Ads publication dates range:', df['pub_datum'].min(), '-', df['pub_datum'].max(), '\n')
print('Missing data by column:', '\n', df.isnull().sum(), '\n')

Number of records: 5000486 

Number of unique user IDs: 782084 

Number of unique job IDs: 63832 

Ads click dates range: 2021-10-28 00:00:00 - 2021-12-15 00:00:00 

Ads publication dates range: 2021-04-01 14:18:42 - 2021-12-14 00:03:55 

Missing data by column: 
 userId              0
jobId               0
dateYM              0
nsession            0
loggedIn            0
place             684
zip           1149852
title               0
sector              0
workregion         27
pub_datum           0
dtype: int64 



In [6]:
# Fill missing values by mapping work region-place columns
mapping = (df[df['place'].notnull()].drop_duplicates('workregion').set_index('workregion').place)
mapping

workregion
:5:11:                                    Region Zürich
:10:                                              Basel
:4:2:                                          Wetzikon
:3:                                              Zürich
:11:                                                Zug
                               ...                     
:14:3:                                           Dublin
:14:22:        St.Gallen (CH)/Freiburg im Breisgau (DE)
:15:18:    Home-Office, Region Oberwallis oder Freiburg
:18:8:                                          Kerzers
:14:23:                                         Balzers
Name: place, Length: 358, dtype: category
Categories (5261, object): ['(Vaduz/Triesen)', '-', '.', '.Zürich', ..., 'Östlichen Raum Zürich, AR, SG, GB, GL, LIE', 'Östlicher Teil des Kantons Aargau', 'à Genève', 'à Niederwangen (BE)']

In [7]:
# Fill missing values in "place" by mapping "workregion"
df['place'] = df['workregion'].map(mapping).fillna(np.nan)

# Print basic details
print('Number of records:', df.shape[0], '\n')
print('Number of unique user IDs:', df['userId'].nunique(), '\n')
print('Number of unique job IDs:', df['jobId'].nunique(), '\n')
print('Ads click dates range:', df['dateYM'].min(), '-', df['dateYM'].max(), '\n')
print('Ads publication dates range:', df['pub_datum'].min(), '-', df['pub_datum'].max(), '\n')
print('Missing data by column:', '\n', df.isnull().sum(), '\n')

Number of records: 5000486 

Number of unique user IDs: 782084 

Number of unique job IDs: 63832 

Ads click dates range: 2021-10-28 00:00:00 - 2021-12-15 00:00:00 

Ads publication dates range: 2021-04-01 14:18:42 - 2021-12-14 00:03:55 

Missing data by column: 
 userId              0
jobId               0
dateYM              0
nsession            0
loggedIn            0
place              10
zip           1149852
title               0
sector              0
workregion         27
pub_datum           0
dtype: int64 



### Duplicates
- The data sample doesn't have duplicated records.
- However, when looking at duplicates for columns subsets (excluding only ad view date and number of sessions per day), it's interesting to notice that 426'157 events are constituted by the same users that visit the same job ad more than once, in different dates.

In [ ]:
# Check duplicates on entire columns
df.duplicated().sum()

In [ ]:
# Check duplicates excluding date and number of sessions
df.duplicated(subset=['userId', 'jobId', 'loggedIn', 'place', 'zip', 'title', 'sector', 'workregion', 'pub_datum']).sum()

### Encoding and feature engineering

In [8]:
# Encode string features
df['loggedIn_num'] = pd.get_dummies(df['loggedIn'], drop_first=True)
df['place_num']= df['place'].cat.codes
df['sector_num'] = df['sector'].cat.codes

# Adjust date format publication date
df['pub_date'] = pd.to_datetime(df['pub_datum']).dt.date
df['pub_date'] = pd.to_datetime(df['pub_date'])

## Create new features
# Days passed between publication and visit
df['pub_visit_days'] = (df['dateYM'] - df['pub_date']).dt.days

## 2) Exploratory Data Analysis

### 2.1) How many clicks does a user make? In how many different regions?

In [9]:
# Groupby userId
user_g = df.groupby('userId').agg({'jobId': ['count', 'nunique'], # Number of clicks per user, Number of clicks on unique ads per user
                                   'nsession': 'sum',  # Total number of sessions
                                   'place': 'nunique', # Number of unique regions clicked
                                   'sector': 'nunique', # Number of unique sectors clicked
                                   'title': 'nunique', # Number of unique job titles clicked
                                   'pub_visit_days' : 'mean' # Average number of days between publication and click
                                  })

display(user_g)
display(round(user_g.describe(), 2))

jobId         nsession   place  sector  \
                                     count nunique      sum nunique nunique   
userId                                                                        
00000000-2966-46d5-ffff-ffffc41afe34     1       1        1       1       1   
00000000-2966-bb95-ffff-ffffb02a6c56     6       6        6       5       5   
00000000-31e3-9de1-0000-00001c908cca     2       2        2       2       2   
00000000-31e3-9de1-0000-000068f0c4fb     1       1        1       1       1   
00000a72-5e0b-4b32-a7b8-9be1a6b2884e     2       2        2       1       1   
...                                    ...     ...      ...     ...     ...   
ffffffff-f636-456a-0000-000047d5e7d7     3       3        3       3       2   
ffffffff-f636-456a-0000-0000641540eb     2       2        2       2       1   
ffffffff-f636-456a-0000-00006d8e5aa7     1       1        1       1       1   
ffffffff-f636-456a-ffff-ffffd4097768     2       2        2       2       2   
ffffffff-f636-456a-ffff-ffffd5d47ee1    97      76       97      16      17   

                                       title pub_visit_days  
                                     nunique           mean  
userId                                                       
00000000-2966-46d5-ffff-ffffc41afe34       1       0.000000  
00000000-2966-bb95-ffff-ffffb02a6c56       6       6.500000  
00000000-31e3-9de1-0000-00001c908cca       2      16.000000  
00000000-31e3-9de1-0000-000068f0c4fb       1       6.000000  
00000a72-5e0b-4b32-a7b8-9be1a6b2884e       2       4.000000  
...                                      ...            ...  
ffffffff-f636-456a-0000-000047d5e7d7       3      11.333333  
ffffffff-f636-456a-0000-0000641540eb       2      15.500000  
ffffffff-f636-456a-0000-00006d8e5aa7       1       5.000000  
ffffffff-f636-456a-ffff-ffffd4097768       2      16.000000  
ffffffff-f636-456a-ffff-ffffd5d47ee1      70       2.082474  

[782084 rows x 7 columns]

jobId              nsession      place     sector      title  \
           count    nunique        sum    nunique    nunique    nunique   
count  782084.00  782084.00  782084.00  782084.00  782084.00  782084.00   
mean        6.39       5.85       6.63       2.55       2.63       5.75   
std        31.29      26.64      33.63       3.99       3.44      24.95   
min         1.00       1.00       1.00       1.00       1.00       1.00   
25%         1.00       1.00       1.00       1.00       1.00       1.00   
50%         1.00       1.00       1.00       1.00       1.00       1.00   
75%         4.00       3.00       4.00       2.00       3.00       3.00   
max     16186.00   13638.00   18235.00     207.00      25.00   12265.00   

      pub_visit_days  
                mean  
count      782084.00  
mean           11.60  
std             8.63  
min           -44.00  
25%             4.50  
50%            11.00  
75%            18.83  
max           240.00

### The data presents ouliers, so it must be cleaned removing observations with z-score > 3

In [10]:
from scipy import stats

# Filter for outliers 
#user_filt = user_g[(np.abs(stats.zscore(user_g[('jobId', 'count')])) < 3)]

user_filt = user_g[(np.abs(stats.zscore(user_g)) < 3).all(axis=1)]
non_outliers = list(user_filt.index)
display(round(user_filt.describe(),2))

jobId              nsession      place     sector      title  \
           count    nunique        sum    nunique    nunique    nunique   
count  751129.00  751129.00  751129.00  751129.00  751129.00  751129.00   
mean        3.37       3.17       3.48       1.98       2.10       3.15   
std         5.32       4.93       5.55       1.99       2.13       4.86   
min         1.00       1.00       1.00       1.00       1.00       1.00   
25%         1.00       1.00       1.00       1.00       1.00       1.00   
50%         1.00       1.00       1.00       1.00       1.00       1.00   
75%         3.00       3.00       3.00       2.00       2.00       3.00   
max       100.00      81.00     107.00      14.00      12.00      79.00   

      pub_visit_days  
                mean  
count      751129.00  
mean           11.77  
std             8.57  
min           -14.30  
25%             4.50  
50%            11.00  
75%            19.00  
max            37.40

### 2.2) Which are the most clicked jobs, sectors and work regions?

In [12]:
print('Clicks events (excluding userId outliers):', len(df[df['userId'].isin(non_outliers)]), '\n')
print('Unique userId (excluding userId outliers):', df[df['userId'].isin(non_outliers)]['jobId'].nunique(), '\n')
print('Clicks events (with userId outliers):', len(df), '\n')

Clicks events (excluding userId outliers): 2529129 

Clicks events (with userId outliers): 5000486 



### Most clicked job titles

In [ ]:
## Top 10 countries by ESG score bar chart

data = df[df['userId'].isin(non_outliers)]['title'].value_counts().head(10).values
lbls = df[df['userId'].isin(non_outliers)]['title'].value_counts().head(10).index

# Plot
plt.style.use('ggplot')#set style
plt.rcParams["font.size"] = 11 # set default font size

fig = plt.figure(figsize=(15, 8))#set subplot and fig size

plt.bar(np.arange(10), data, color=sns.color_palette("plasma", 10))
plt.xticks(np.arange(10), lbls, rotation=45, ha="right")
plt.ylabel('Number')
#plt.title('Top 10 countries by ESG score')

plt.show()

In [13]:
df[df['userId'].isin(non_outliers)]['title'].value_counts().head(20)

Elektro-Projektleiter 80 - 100% (m/w)                                           152498
Sachbearbeiter (100% m/w) Quereinstieg Finanzen neue Herausforderung             39047
Junior-Projektleiter Elektroinstallationen (m/w)                                 18726
Sachbearbeiter Familienzulagen (w/m) 80 - 100%                                   14880
Stv. Bauleitender Elektroinstallateur 80 - 100% (m/w)                             8526
Medical Performance Manager 50% - 70%                                             6123
Netzelektriker (m/w)                                                              5574
Applikationsverantwortliche/r KIS &amp; Umsysteme 100%                            4720
Mitarbeiterin Verkauf und Service 50%                                             4681
Immobilienmakler/in                                                               4516
Mitarbeiter Gepäcksortierung & Flugzeugabfertigung 20-80%                         4211
Immobilienbewirtschafter/in mit Eidg. FA   

In [ ]:
df['title'].value_counts().head(20)

In [ ]:
sectors = df[df['userId'].isin(non_outliers)]['sector'].value_counts().head(10).to_frame().reset_index()
sectors.rename(columns={'index': 'Sectors', 'sector': 'Clicks'}, inplace=True)
sectors

### Most clicked sectors

In [14]:
df[df['userId'].isin(non_outliers)]['sector'].value_counts().head(10)

jobsse-2     346132
jobsse-12    293746
jobsse-10    279266
jobsse-7     199148
jobsse-6     192519
jobsse-16    177032
jobsse-0      98348
jobsse-1      96096
jobsse-13     93637
jobsse-18     90420
Name: sector, dtype: int64

In [ ]:
df['sector'].value_counts().head(10)

In [20]:
df[df['sector'] == 'jobsse-12']['title'].value_counts().head(20)

Sachbearbeiter (100% m/w) Quereinstieg Finanzen neue Herausforderung          39968
Fachspezialist Steuerung 100% (m/w)                                            3264
Quereinsteiger als Verkaufsberater Innen-& Aussendienst 80-100% (m/w)          1448
Recruiting Partner (80-100%)                                                   1389
Mitarbeiter Empfang und Administration w/m (40%)                               1176
Mitarbeiter Produktion (m/w)                                                   1129
Sachbearbeiter technischer Kundenservice im Pensum von 60 bis 70 % (m/w)       1098
Leiter Operations (m/w)  Mitglied der Geschäftsleitung                         1027
Key Account Manager (m/w) Umweltsensorik                                        999
Sachbearbeiter Service Innendienst PoU (w/m/d)                                  986
SachbearbeiterIn Innendienst 100%                                               958
Projektleiter (m/w) Digital Marketing                                       

### Most clicked regions

In [15]:
df[df['userId'].isin(non_outliers)]['place'].value_counts().head(10)

Zürich         374885
Basel          371878
Zollikofen     308744
Zug            221844
Brügg          201475
Wallisellen     73802
Regensdorf      69109
Beringen        69100
Glattbrugg      54380
Renens          48227
Name: place, dtype: int64

In [ ]:
df['place'].value_counts().head(30)

### 2.3) Are there differences between logged-in users and anonymous sessions?

In [17]:
# Groupby loggedIn
logged_g = df[df['userId'].isin(non_outliers)].groupby(['loggedIn']).agg({'loggedIn': 'count',
                                                                          'userId': 'nunique',
                                                                          'jobId' :'nunique',
                                                                          'nsession' : 'mean',
                                                                          'title': 'nunique',
                                                                          'sector': 'nunique',
                                                                          'pub_visit_days' : 'mean'}).sort_values('nsession', ascending=False)


logged_g['avg_click_per_user'] = logged_g['loggedIn']/logged_g['userId']
logged_g['avg_unique_job_per_user'] = logged_g['jobId']/logged_g['userId']

display(round(logged_g, 2))

,loggedIn,userId,jobId,nsession,title,sector,pub_visit_days,avg_click_per_user,avg_unique_job_per_user
loggedIn,,,,,,,,,
True,516196,43147,56528,1.05,42088,25,6.50,11.96,1.31
False,2012933,707982,29735,1.03,25593,25,10.95,2.84,0.04


In [19]:
logged_g.rename(columns={'loggedIn': 'Clicks', 'userId': 'Unique_users'}, inplace=True)
logged_ch = logged_g.reset_index()[['loggedIn', 'Clicks', 'Unique_users', 'nsession', 'pub_visit_days', 'avg_click_per_user']]
logged_ch

,loggedIn,Clicks,Unique_users,nsession,pub_visit_days,avg_click_per_user
0,True,516196,43147,1.052767,6.501027,11.963659
1,False,2012933,707982,1.030051,10.954522,2.843198


### A/B test logged in vs. anonymous

In [21]:
# Define groups authenticated users vs. anonimous
# and calculate the average number of clicks

login_users = df[(df['userId'].isin(non_outliers)) & (df['loggedIn'] == True)].groupby('userId').agg({'jobId': ['count', 'nunique'], # Number of clicks per user, Number of clicks on unique ads per user
                                   'nsession': 'sum',  # Total number of sessions
                                   'place': 'nunique', 'pub_visit_days' : 'mean'  # Number of unique regions
                                  })



anonym_users = df[(df['userId'].isin(non_outliers)) & (df['loggedIn'] == False)].groupby('userId').agg({'jobId': ['count', 'nunique'], # Number of clicks per user, Number of clicks on unique ads per user
                                   'nsession': 'sum',  # Total number of sessions
                                   'place': 'nunique', 'pub_visit_days' : 'mean'  # Number of unique regions
                                  })



In [22]:
import scipy.stats as stats

# Perform two-sided Mann-Whitney U rank test
print('Difference in loggedIn/anonymous average number of clicks:', '\n',
      stats.mannwhitneyu(x=login_users[('jobId', 'count')], y=anonym_users[('jobId', 'count')], alternative = 'two-sided'), '\n')
print('Difference in loggedIn/anonymous click delay from publication:', '\n',
      stats.mannwhitneyu(x=login_users[('pub_visit_days', 'mean')], y=anonym_users[('pub_visit_days', 'mean')], alternative = 'two-sided'))

Difference in loggedIn/anonymous average number of clicks: 
 MannwhitneyuResult(statistic=25798898879.5, pvalue=0.0) 

Difference in loggedIn/anonymous click delay from publication: 
 MannwhitneyuResult(statistic=9867877076.0, pvalue=0.0)


In [ ]:
ax = sns.histplot(x=anonym_users[('pub_visit_days', 'mean')], kde=True)

In [ ]:
ax = sns.boxplot(x=anonym_users[('jobId', 'count')])

In [ ]:
from scipy import stats


#login_users_norm = ((login_users-login_users.mean())/login_users.std())
#anonym_users_norm = ((anonym_users-anonym_users.mean())/anonym_users.std())

ax = sns.histplot(x=login_users[('pub_visit_days', 'mean')], kde=True)

In [ ]:
from scipy.stats import ttest_ind

ttest_ind(login_users[('pub_visit_days', 'mean')], anonym_users[('pub_visit_days', 'mean')], equal_var=False)